# WeRateDogs Tweeter Archive

## Gather

In [1]:
import pandas as pd
import numpy as np
import requests
#import tweepy

In [2]:
tweet = pd.read_csv('twitter-archive-enhanced.csv')

In [ ]:
###
r = requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')
if r.status_code == 200:
    with open('image_predictions.tsv', 'wb') as f:
        f.write(r.content)

In [4]:
image = pd.read_csv('image_predictions.tsv', sep = '\t')

In [ ]:
###
consumer_key = 'X4Mu5bFyMBpjjstVS4XbYjZcL'
consumer_secret = '6nXJBPj6kBPzT0zP45fZnjiAOuMTN5OEBS3tQ7xKbfIT4NuIst'
access_token = '4366384514-l6MKA0qiPnv018rq4aSQPQd5TODG6Ko4gNeBlSA'
access_secret = '6HcJzhQoY9fZaTfj8p5XXyBdCjNsKiB8SUURkRmSQEh3H'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True )

In [ ]:
###
from timeit import default_timer as timer

start = timer()
tweet_id = []
retweet_count = []
favorite_count = []

for ids in image.tweet_id:
    try:
        tweet = api.get_status(ids, tweet_mode='extended')
        tweet_id.append(ids)
        retweet_count.append(tweet.retweet_count)
        favorite_count.append(tweet.favorite_count)
    except Exception as e:
        print(str(e))
        
end = timer()
print(end - start)    

In [ ]:
###
file = open("tweet_json.txt", "w")
for index in range(len(tweet_id)):
    file.write(str(tweet_id[index]) + " " + str(retweet_count[index]) + " " + str(favorite_count[index]) + "\n")
file.close()

In [5]:
info = pd.read_csv('tweet_json.txt', sep=" ", header=None)
info.columns = ['tweet_id','retweet_count','favorite_count']

## Assess

In [ ]:
tweet.head(10)

In [ ]:
image.head(10)

In [ ]:
info.head(10)

In [ ]:
tweet.info()

In [ ]:
image.info()

In [ ]:
info.info()

In [ ]:
all_columns = pd.Series(list(tweet) + list(image) + list(info))
all_columns[all_columns.duplicated()]

In [ ]:
list(tweet)

In [ ]:
list(image)

In [ ]:
tweet[tweet['expanded_urls'].isnull()]

In [ ]:
tweet.describe()

In [ ]:
image.describe()

In [ ]:
info.describe()

In [ ]:
tweet.sample(5)

In [ ]:
tweet.name.value_counts()

In [ ]:
tweet.source.value_counts()

In [ ]:
image.jpg_url.value_counts()

In [ ]:
tweet[tweet.name.duplicated()]

In [ ]:
tweet.rating_numerator.sort_values()

In [ ]:
tweet.rating_denominator.sort_values()

In [ ]:
len(tweet[(tweet['doggo'] == 'doggo')]) + len(tweet[(tweet['floofer'] == 'floofer')])  + len(tweet[(tweet['pupper'] == 'pupper')]) 
+ len(tweet[(tweet['puppo'] == 'puppo')]) 

#### Quality
##### `tweet` table
- in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, and retweeted_status_user_id are int not float
- Erroneous datatypes (retweeted_status_timestamp, timestamp, text, and source columns) 
- source has four categories
- name sometimes lower case, captial letter other times
- Missing expanded_urls (2297 instead of 2356)
- Missing in_reply_to_status_id and in_reply_to_user_id (78 instead of 2356)
- Missing retweeted_status_id, retweeted_status_id, retweeted_status_id (181 instead of 2356)
- actually, by, his, my, one, the, very are not dog name
- Multiple records for 'a, an' as a name
- Some rating_numerator and rating_denominator are extremely high
- Some rows have rating_numerator and rating_denominator but do not have expanded_urls

##### `image` table
- Duplicates in jpg_url
- Lowercase p1, p2, p3
- Missing records (2075 instead of 2356)

##### `info` table
- Missing records (2069 instead of 2075)

#### Tidiness
- Merge doggo, pupper, puppo, and floofer columns into one column in `tweet` table
- `info` table should be part of the `image` table

## Clean

In [26]:
tweet_clean = tweet.copy()
image_clean = image.copy()
info_clean = info.copy()

### Missing Data

#### `tweet`: Missing expanded_urls (2297 instead of 2356), Missing in_reply_to_status_id and in_reply_to_user_id (78 instead of 2356), Missing retweeted_status_id, retweeted_status_id, retweeted_status_id (181 instead of 2356)

##### Define
- Remove tweets that don't have image url
- Remove replied tweets
- Remove retweets

##### Code

In [ ]:
tweet_clean = tweet_clean[~tweet_clean.expanded_urls.isnull()]

##### Test

In [ ]:
tweet_clean.head()

##### Code

In [ ]:
tweet_clean = tweet_clean[tweet_clean['in_reply_to_status_id'].isnull()]

##### Test

In [ ]:
len(tweet_clean)

##### Code

In [ ]:
tweet_clean = tweet_clean[tweet_clean['retweeted_status_id'].isnull()]

##### Test

In [ ]:
len(tweet_clean)

##### Code

In [ ]:
drop_columns = ['in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 
                'retweeted_status_user_id', 'retweeted_status_timestamp']

tweet_clean = tweet_clean.drop(labels = drop_columns, axis = 1)

##### Test

In [ ]:
tweet_clean.head()

### Quality

#### `tweet`: Erroneous datatypes (tweet_id, timestamp, text, and source columns) , Source has four categories

##### Define
- Change tweet_id to int data type
- Change timestamp to date data type
- Change text to string data type
- Change source to category data type
- Replace the four category source with short version

##### Code

In [ ]:
tweet_clean.tweet_id = tweet_clean.tweet_id.astype(int) 
tweet_clean.text = tweet_clean.text.astype(str)
tweet_clean.source = tweet_clean.source.astype('category')
tweet_clean.timestamp = pd.to_datetime(tweet_clean.timestamp)

In [ ]:
tweet_clean.source = tweet_clean.source.replace('<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'Twitter for iPhone')
tweet_clean.source = tweet_clean.source.replace('<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>', 'Vine Make a Scene')
tweet_clean.source = tweet_clean.source.replace('<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>', 'Twitter Web Client')
tweet_clean.source = tweet_clean.source.replace('<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>', 'Tweet Deck')

##### Test

In [ ]:
tweet_clean.info()

In [ ]:
tweet_clean.head()

#### `tweet`: Some rating_numerator and rating_denominator are extremely high

##### Define
- Calculate the rate based on rating_numerator divide rating_denominator
- Remove the rates greater than 2.0

##### Code

In [ ]:
tweet_clean['rate'] = tweet_clean.rating_numerator / tweet_clean.rating_denominator

In [ ]:
for dog_rate in tweet_clean.rate:
    if dog_rate > 2.0:
        print(dog_rate)

In [ ]:
tweet_clean = tweet_clean[tweet_clean.rate < 2]

##### Test

In [ ]:
tweet_clean.rate.sort_values()

#### `tweet`: Multiple records for 'a, an' as a name, 'actually, by, his, my, one, the, very' are not dog name

##### Define
- Filter the text column to check if 'a','an', etc have corresponding name
- If not, change the name to 'None'

##### Code

In [7]:
Exception_names = ['a', 'an','actually', 'by', 'his', 'my', 'one', 'the', 'very']

In [27]:
for names in tweet_clean.name:
    if names in Exception_names:
#            print(names)
            tweet_clean['new_text'] = tweet_clean['text'].loc[tweet_clean['name'] == names]
#             df_text.append(tweet_clean['text'].loc[tweet_clean['name'] == names])
#         print(tweet_clean.loc[tweet_clean['name'] == names, 'text'])
# for descrption in tweet_clean.text:
#     print(descrption)

In [ ]:
tweet_clean['name'].value_counts()

In [ ]:
tweet_clean['new_text'][~tweet_clean['new_text'].isnull()]

In [28]:
# tweet_clean['new_text'].str.split('\\bThis is a \\b')
tweet_clean['new_text'] = tweet_clean['new_text'].str.extract('\This is a (.*?)\.').str.split().str[-1]

/Users/Miyal/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  from ipykernel import kernelapp as app


In [21]:
tweet_clean['new_text']

0               NaN
1               NaN
2               NaN
3               NaN
4               NaN
5               NaN
6               NaN
7               NaN
8               NaN
9               NaN
10              NaN
11              NaN
12              NaN
13              NaN
14              NaN
15              NaN
16              NaN
17              NaN
18              NaN
19              NaN
20              NaN
21              NaN
22              NaN
23              NaN
24              NaN
25              NaN
26              NaN
27              NaN
28              NaN
29              NaN
           ...     
2326            NaN
2327    bumblegruff
2328            NaN
2329            NaN
2330            NaN
2331            NaN
2332            NaN
2333            NaN
2334            dog
2335            NaN
2336            NaN
2337            NaN
2338            NaN
2339            NaN
2340            NaN
2341            NaN
2342            NaN
2343            NaN
2344            NaN


In [30]:
tweet_clean['new_text'] = tweet_clean['new_text'].fillna(tweet_clean['name'])

In [31]:
tweet_clean.new_text

0           Phineas
1             Tilly
2            Archie
3             Darla
4          Franklin
5              None
6               Jax
7              None
8              Zoey
9            Cassie
10             Koda
11            Bruno
12             None
13              Ted
14           Stuart
15           Oliver
16              Jim
17             Zeke
18          Ralphus
19           Canela
20           Gerald
21          Jeffrey
22             such
23           Canela
24             None
25             None
26             Maya
27           Mingus
28            Derek
29           Roscoe
           ...     
2326          quite
2327    bumblegruff
2328           None
2329           None
2330           None
2331           None
2332           None
2333             an
2334            dog
2335             an
2336           None
2337           None
2338           None
2339           None
2340           None
2341           None
2342           None
2343           None
2344           None


##### Test

#### `image`: Duplicates in jpg_url

##### Define
- Find the duplicates in jpg_url
- Remove the duplicate rows

##### Code

In [ ]:
image_clean = image_clean[~image_clean.duplicated(['jpg_url'], keep=False)]

##### Test

In [ ]:
image_clean.info()

#### `tweet`: Name sometimes lower case, captial letter other times
#### `image`: Lowercase p1, p2, p3

##### Define
- Lower case name column in 'tweet' table
- Lower case p1, p2, p3 columns in 'image' table

##### Code

In [ ]:
tweet_clean['name'] = tweet_clean['name'].str.lower()
image_clean['p1'] = image_clean['p1'].str.lower()
image_clean['p2'] = image_clean['p2'].str.lower()
image_clean['p3'] = image_clean['p3'].str.lower()

##### Test

In [ ]:
tweet_clean.sample(5)

In [ ]:
image_clean.sample(5)

### Tidiness

#### Merge doggo, pupper, puppo, and floofer columns into one column in `tweet` table

##### Define
*Your definition here. Hint: use pandas' [melt function](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.melt.html) and [`str.split()` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.split.html). Here is an excellent [`melt` tutorial](https://deparkes.co.uk/2016/10/28/reshape-pandas-data-with-melt/).*

Melt the auralin and novodra columns to a treatment and a dose column (dose will still contain both start and end dose at this point). Then split the dose column on ' - ' to obtain start_dose and end_dose columns. Drop the intermediate dose column.

##### Code

In [ ]:
# Your cleaning code here
treatments_clean = pd.melt(treatments_clean, id_vars=['given_name', 'surname', 'hba1c_start', 'hba1c_end', 'hba1c_change'],
                           var_name='treatment', value_name='dose')
treatments_clean = treatments_clean[treatments_clean.dose != "-"]
treatments_clean['dose_start'], treatments_clean['dose_end'] = treatments_clean['dose'].str.split(' - ', 1).str
treatments_clean = treatments_clean.drop('dose', axis=1)

##### Test

In [ ]:
# Your testing code here
treatments_clean.head()

#### `info` table should be part of the `image` table

##### Define
Merge the info table to the treatments table, joining on tweet_id.

##### Code

In [ ]:
image_clean = pd.merge(image_clean, info_clean,
                            on=['tweet_id'], how='left')

##### Test

In [ ]:
image_clean.info()

In [ ]:
image_clean[image_clean['retweet_count'].isnull()]

In [ ]:
image_clean.sample(5)

### Quality

<font color='red'>Complete the remaining "Quality" **Define, Code, and Test** sequences after watching the *"Cleaning for Quality"* video.</font>

#### Multiple phone number formats

##### Define
*Your definition here. Hint: helpful [Stack Overflow answer](https://stackoverflow.com/a/123681).*

Strip all " ", "-", "(", ")", and "+" and store each number without any formatting. Pad the phone number with a 1 if the length of the number is 10 digits (we want country code).

##### Code

In [ ]:
# Your cleaning code here
patients_clean.phone_number = patients_clean.phone_number.str.replace(r'\D+', '').str.pad(11, fillchar='1')

##### Test

In [ ]:
# Your testing code here
patients_clean.phone_number.head()

#### Default John Doe data

##### Define
*Your definition here. Recall that it is assumed that the data that this John Doe data displaced is not recoverable.*

Remove the non-recoverable John Doe records from the patients table.

##### Code

In [ ]:
# Your cleaning code here
patients_clean = patients_clean[patients_clean.surname != 'Doe']

##### Test

In [ ]:
# Your testing code here
# Should be no Doe records
patients_clean.surname.value_counts()

In [ ]:
# Should be no 123 Main Street records
patients_clean.address.value_counts()

#### Multiple records for Jakobsen, Gersten, Taylor

##### Define
*Your definition here.*

Remove the Jake Jakobsen, Pat Gersten, and Sandy Taylor rows from the patients table. These are the nicknames, which happen to also not be in the treatments table (removing the wrong name would create a consistency issue between the patients and treatments table). These are all the second occurrence of the duplicate. These are also the only occurences of non-null duplicate addresses.

##### Code

In [ ]:
# Your cleaning code here
# tilde means not: http://pandas.pydata.org/pandas-docs/stable/indexing.html#boolean-indexing
patients_clean = patients_clean[~((patients_clean.address.duplicated()) & patients_clean.address.notnull())]

##### Test

In [ ]:
# Your testing code here
patients_clean[patients_clean.surname == 'Jakobsen']

In [ ]:
patients_clean[patients_clean.surname == 'Gersten']

In [ ]:
patients_clean[patients_clean.surname == 'Taylor']